In [1]:
"""
Requirements: 
* One Generate Patches script to take in dataset and make patches out of them and store them into designated location in datasets folder
* train test split
* One dataloader for training and one dataloader for testing.

* Construct the Unet Model. Take care of input size, take care that you may need to incorporate grid and inverse grid function later.
    * Unet model should have functionalities: 
      * Feed data 
      * ... all functionalities as available in image restoration model. But make it only essential.

* Write a training script as well. 
"""

'\nRequirements: \n* One Generate Patches script to take in dataset and make patches out of them and store them into designated location in datasets folder\n* train test split\n* One dataloader for training and one dataloader for testing.\n\n* Construct the Unet Model. Take care of input size, take care that you may need to incorporate grid and inverse grid function later.\n    * Unet model should have functionalities: \n      * Feed data \n      * ... all functionalities as available in image restoration model. But make it only essential.\n\n* Write a training script as well. \n'

# Generating Patches

In [27]:
import cv2 
import numpy as np 
import random 
import os 
from tqdm.auto import tqdm
from glob import glob

In [31]:
LR_images = './datasets/raw/Train/LR_x4/' 
HR_images = './datasets/raw/Train/HR/'
SR_images = './datasets/raw/Train/SR/'

if not os.path.exists('./datasets/patches/'): 
    os.makedirs('./datasets/patches') 
    os.makedirs('./datasets/patches/HR')
    os.makedirs('./datasets/patches/SR') 
else: 
    print("patches Already present")

In [32]:
crop_h, crop_w = 256, 256
stride = 100 

N_T = len(os.listdir(HR_images))//2 
patches_hr = './datasets/patches/HR'
patches_sr = './datasets/patches/SR'

img_fmt = '0000{}_{}.png'
idx_patch = 1 
folder_name_fmt = '000000{}'
for i in tqdm(range(1, N_T+1)): 
    limg, rimg = img_fmt.format(i,'L'), img_fmt.format(i,'R') 
    limg, rimg = limg[-10:], rimg[-10:]
    hr_l = cv2.imread(os.path.join(HR_images,limg)) 
    hr_r = cv2.imread(os.path.join(HR_images,rimg))  

    sr_l = cv2.imread(os.path.join(SR_images,limg)) 
    sr_r = cv2.imread(os.path.join(SR_images,rimg)) 

    # print(limg, rimg, '\r') 
    H = hr_l.shape[0] 
    W = hr_l.shape[1]

    for y in range(0, H-crop_h+1, stride):  
        for x in range(0, W-crop_w+1, stride): 
            patch_hr_l = hr_l[y:y+crop_h, x:x+crop_w] 
            patch_hr_r = hr_r[y:y+crop_h, x:x+crop_w]
            dest_folder = os.path.join(patches_hr, folder_name_fmt.format(idx_patch)[-6:])
            os.makedirs(dest_folder, exist_ok=True)
            cv2.imwrite(os.path.join(dest_folder,'hr_l.png'), patch_hr_l)
            cv2.imwrite(os.path.join(dest_folder,'hr_r.png'), patch_hr_r)

            patch_sr_l = sr_l[y:y+crop_h, x:x+crop_w]
            patch_sr_r = sr_r[y:y+crop_h, x:x+crop_w]
            dest_folder = os.path.join(patches_sr, folder_name_fmt.format(idx_patch)[-6:]) 
            os.makedirs(dest_folder, exist_ok=True) 
            cv2.imwrite(os.path.join(dest_folder,'sr_l.png'), patch_sr_l)
            cv2.imwrite(os.path.join(dest_folder,'sr_r.png'), patch_sr_r)


            idx_patch += 1


    # break 

  0%|          | 0/800 [00:00<?, ?it/s]

# Data Loaders

In [33]:
import os 
from torch.utils.data import Dataset 
from PIL import Image

class CustomLoader(Dataset): 
    def __init__(self, hr_dir, sr_dir, list_idxs, transform = None):
        """
        returns hr_l, hr_r, sr_l, sr_r
        """
        self.hr_dir, self.sr_dir = hr_dir , sr_dir 
        self.list_idxs = list_idxs 
        N = len(list_idxs)
        self.idx_map = {i:list_idxs[i] for i in range(N)} 
        self.folder_name = '000000{}'.format(idx)[-6:]

    def __len__(self): 
        return len(self.list_idxs)

    def __getitem__(self, index): 
        idx = self.idx_map[index] 
         

        hr_l = os.path.join(self.hr_dir,self.folder_name,'hr_l.png')
        hr_r = os.path.join(self.hr_dir,self.folder_name,'hr_r.png')
        hr_l = np.array(Image.open(hr_l).convert("RGB")) 
        hr_r = np.array(Image.open(hr_r).convert("RGB"))

        sr_l = os.path.join(self.sr_dir,self.folder_name,'sr_l.png') 
        sr_r = os.path.join(self.sr_dir,self.folder_name,'sr_r.png')
        sr_l = np.array(Image.open(sr_l).convert("RGB"))
        sr_r = np.array(Image.open(sr_r).convert("RGB"))

        if self.transform is not None: 
            pass 

        return (hr_l,hr_r,sr_l,sr_r)


        

ModuleNotFoundError: No module named 'torch'